### Extracting all Videos Ids from the Uploads playlist

In [73]:
channelid='UCZSNzBgFub_WWil6TOTYwAg'
vpart='contentDetails,id,liveStreamingDetails,localizations,player,snippet,statistics,status,topicDetails'

In [27]:
from apiclient.discovery import build

In [28]:
api_key="xxxxxxxxxxxxxxxxxxxxxx" #add API key
youtube = build('youtube', 'v3', developerKey=api_key)

In [37]:
def get_channel_videos(channel_id):
    
    # get Uploads playlist id
    res = youtube.channels().list(id=channel_id, 
                                  part='contentDetails').execute()
    playlist_id = res['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id, 
                                           part="id,snippet",
                                           maxResults=50,
                                           pageToken=next_page_token).execute()
        videos += res['items']
        next_page_token = res.get('nextPageToken')
        
        if next_page_token is None:
            break
    
    return videos

In [38]:
videos=get_channel_videos(channel_id=channelid)

### Video Details for every video using it's Id

In [64]:
netflixyoutube=[]
for video in videos:
  res = youtube.videos().list(part=vpart,id=video['snippet']['resourceId']['videoId']).execute()
  netflixyoutube.append(res)

In [71]:
import json
with open('netflixyoutube.json', 'w') as f:
    json.dump(netflixyoutube, f)

### Analysis on the data

In [1]:
import json
with open('data.json', 'r') as f:
    data=json.load(f)

In [2]:
title=[]
views=[]
duration=[]
id=[]
time=[]
tags=[]
likes=[]
dislikes=[]
comments=[]
description=[]
thumbnails=[]

In [3]:

for v in data:
  duration.append(v['items'][0]['contentDetails']['duration'])
  id.append(v['items'][0]['id'])
  description.append(v['items'][0]['snippet']['description'])
  title.append(v['items'][0]['snippet']['title'])
  time.append(v['items'][0]['snippet']['publishedAt'])
  try:
    tags.append(v['items'][0]['snippet']['tags'])
  except KeyError:
    tags.append([''])
  views.append(v['items'][0]['statistics']['viewCount'])
  try:
    comments.append(v['items'][0]['statistics']['commentCount'])
  except KeyError:
    comments.append('')
  try:
    likes.append(v['items'][0]['statistics']['likeCount'])
  except KeyError:
    likes.append('')
  try:
    dislikes.append(v['items'][0]['statistics']['dislikeCount'])
  except:
    dislikes.append('')
  thumbnails.append(v['items'][0]['snippet']['thumbnails'])

    

### Splitting time stamp into date and time

In [5]:
videodate=[]
videotime=[]
for t in time:
  spl=t.split('T')
  date=spl[0]
  ti=spl[1].split('Z')[0]
  videodate.append(date)
  videotime.append(ti)

  

In [6]:
import pandas as pd
df=pd.DataFrame()


In [7]:
df["title"]=title
df["views"]=views
df["duration"]=duration
df["id"]=id
df["time"]=videotime
df["tags"]=tags
df["likes"]=likes
df["dislikes"]=dislikes
df["comments"]=comments
df["description"]=description
df["date"]=videodate
df["thumbnails"]=thumbnails

### Top 3 Videos every year

In [51]:
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d') 

In [133]:
df["views"]=df["views"].astype(int)

In [134]:
df_agg = df.groupby([df.date.dt.year]).apply(lambda x: x.sort_values(by=["views"],ascending=False).head(3))

In [135]:
df_agg

title  ...                                         thumbnails
date                                                          ...                                                   
2016 1282    Diwali with Netflix | Ad-free and Uninterrupted  ...  {'default': {'url': 'https://i.ytimg.com/vi/cD...
     1285             Diwali on Netflix Feat. Anurag Kashyap  ...  {'default': {'url': 'https://i.ytimg.com/vi/Si...
     1293                Netflix | Hooked feat. Tanmay Bhatt  ...  {'default': {'url': 'https://i.ytimg.com/vi/ST...
2017 1272  Marvel's Iron Fist | The Final Defender Has Ar...  ...  {'default': {'url': 'https://i.ytimg.com/vi/vC...
     1245          Dynasty | Official Trailer [HD] | Netflix  ...  {'default': {'url': 'https://i.ytimg.com/vi/Uq...
     1276      Serial Cheaters feat Sumeet & Nidhi | Netflix  ...  {'default': {'url': 'https://i.ytimg.com/vi/io...
2018 1107                       Netflix India’s BEST OF 2018  ...  {'default': {'url': 'https://i.ytimg.com/vi/0L...
     1225                               Heating Up | Netflix  ...  {'default': {'url': 'https://i.ytimg.com/vi/as...
     1190              Ghoul - Silent Trailer I Netflix I HD  ...  {'default': {'url': 'https://i.ytimg.com/vi/Kw...
2019 1029  Mighty Little Bheem | Bheem with a Beat | Netf...  ...  {'default': {'url': 'https://i.ytimg.com/vi/U1...
     1022  Mighty Little Bheem | Flower Adventure | Netfl...  ...  {'default': {'url': 'https://i.ytimg.com/vi/Q_...
     1039   Mighty Little Bheem | Official Trailer | Netflix  ...  {'default': {'url': 'https://i.ytimg.com/vi/4q...
2020 558   Bella Ciao Full Song | La Casa De Papel | Mone...  ...  {'default': {'url': 'https://i.ytimg.com/vi/46...
     589   Allu Arjun Boardroom Dance Scene | Ala Vaikunt...  ...  {'default': {'url': 'https://i.ytimg.com/vi/Gl...
     550   Allu Arjun Fight Scene | Ala Vaikunthapurramlo...  ...  {'default': {'url': 'https://i.ytimg.com/vi/TN...
2021 13    Tribhanga | Official Trailer | Kajol, Mithila ...  ...  {'default': {'url': 'https://i.ytimg.com/vi/zV...
     15    Allu Arjun Police Station Fight Scene | Ala Va...  ...  {'default': {'url': 'https://i.ytimg.com/vi/x7...
     11    @Kapil Sharma's Very Official Announcement | N...  ...  {'default': {'url': 'https://i.ytimg.com/vi/PD...

[18 rows x 12 columns]

In [136]:
df_agg.to_csv("top3.csv")

## Some Insights

### Creators/Guests generating the most views/videos

In [ ]:
import regex as re
#Creating a list of creators
creators=[]
for i in range(0,1310):
  if '@' in title[i]:
    l=re.findall("(?<![@\w])@(\w{1,25} (\w*))",title[i])
    for names in l:
      print(names)
      if names[0] not in creators:
        creators.append(names[0])
  

#### List needs to be cleaned

In [206]:
creatorscleaned=["MostlySane","MyMissAnand","MortaL",'Xyaa','Mythpat','DIVINE']
for c in creators:
  if c.split(' ')[0] not in creatorscleaned:
    creatorscleaned.append(c)

In [224]:
missedcreators=["Srishti Dixit",'Menu Please','Abish']
creators=creatorscleaned+missedcreators
creators.remove("Alia Bhatt")

In [228]:
ratio=[]
totalcount=[]
totalview=[]
for c in creators:
  count=0
  totalviews=0
  for i in range(0,1310):
    v=int(views[i])
    t=title[i]
    if c in t:
      count=count+1
      totalviews+=v
  try:
    ratio.append(totalviews/count)
    totalview.append(totalviews)
    totalcount.append(count)
  except ZeroDivisionError:
    ratio.append(0)


In [231]:
creator_performance=pd.DataFrame()
creator_performance["creator"]=creators
creator_performance["total views"]=totalview
creator_performance["video count"]=totalcount
creator_performance["views/video"]=ratio

In [233]:
creator_performance.sort_values(by="views/video")

,creator,total views,video count,views/video
47,Ankur Kashyap,39045,1,3.904500e+04
50,Be YouNick,41440,1,4.144000e+04
41,Melvin Louis,44534,1,4.453400e+04
5,DIVINE,66847,1,6.684700e+04
14,Anshuman Sharma,104295,1,1.042950e+05
44,Tadpatri Talkies,108245,1,1.082450e+05
30,BeerBiceps,383177,3,1.277257e+05
45,Sucharita Tyagi,144403,1,1.444030e+05
48,Khooni Monday,166026,1,1.660260e+05
20,Varun Thakur,1058927,5,2.117854e+05


In [234]:
creator_performance.to_csv("creator_performance.csv")

### A Comparative Analysis on Shows

In [307]:
shows=["Menu Please","Now Memeing","@Tanmay Bhat &","@Tanmay Bhat Reacts","Grandma","Behensplaining","Decode with","f Abish"]

In [308]:
s=[]
showname=[]
for t in df.title:
  if any(show in t for show in shows):
    s.append(True)
  else:
    s.append(False)
df["series"]=s
showdf=df[df.series==True]

In [309]:
for t in showdf.title:
  for show in shows:
    if show in t:
      showname.append(show)
      

In [311]:
showname_clean=[]
for s in showname:
  if "@Tanmay" in s:
    showname_clean.append("Tanmay Reacts")
  elif 'f Abish' in s:
    showname_clean.append("Son of Abish")
  elif 'Decode' in s:
    showname_clean.append("Decode with Dhruv")
  elif "Grandma" in s:
    showname_clean.append("Grandma Reacts")
  else:
    showname_clean.append(s) 

In [312]:
showdf["showname"]=showname_clean

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [313]:
showdf1=showdf.groupby(showdf.showname)

In [321]:
showdf["likes"]=showdf['likes'].astype(int)
showdf["comments"]=showdf['comments'].astype(int)
showdf["dislikes"]=showdf['dislikes'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [351]:
showdf['date'] = pd.to_datetime(showdf['date'],format='%Y-%m-%d')
last5ep=showdf.groupby('showname').apply(lambda x: x.sort_values(by=["date"],ascending=False).head(5))

In [340]:
showdf.to_csv("showdf.csv")

In [347]:
showdf["popularity"]=showdf["likes"]/showdf["views"]

In [354]:
last5ep.to_csv("last5ep.csv")

### Optimal Duration

In [8]:
duration=[]
for v in data:
  duration.append(v['items'][0]['contentDetails']['duration'])
df["duration"]=duration
duration=[]
for d in df.duration:
  d=d.replace('PT','')
  d=d.replace('S','')
  d=d.replace("M",":")
  d=d.replace("H",":")
  duration.append(d)


In [9]:
#fixing shit now ugh
duration1=[]
print(duration)
for d in duration:
  if d.endswith(":"):
    d=d+"00"
  if ":" not in d:
    d="0:"+d
  duration1.append(d)

['2:2', '2:3', '1:8', '2:35', '4:36', '1:50', '20:19', '2:12', '1:57', '2:13', '3:20', '1:13', '2:39', '2:', '4:39', '3:4', '43:5', '33', '2:13', '2:3', '11:4', '56', '1:28', '3:2', '1:49', '2:31', '6:41', '30:40', '11:59', '4:', '1:7', '8:15', '6:33', '3:5', '26:8', '5:2', '2:40', '59', '8:33', '7:35', '3:9', '14:44', '5:3', '4:6', '8:53', '6:24', '3:13', '3:2', '3:55', '2:54', '32:12', '4:10', '6:57', '2:56', '2:7', '41', '7:', '4:8', '41', '3:49', '5:43', '41', '5:45', '30', '11:7', '4:55', '34:36', '2:33', '3:8', '1:58', '2:49', '2:18', '2:25', '6:33', '2:30', '2:44', '1:48', '53:31', '1:59', '2:12', '2:59', '7:32', '33:8', '1:6', '2:51', '3:', '12:25', '2:35', '2:3', '32:56', '2:11', '6:12', '1:11', '7:16', '1:30', '6:1', '23:39', '6:55', '5:58', '9:15', '30:25', '8:53', '4:27', '41', '15:1', '2:13', '1:41', '25:19', '14:42', '3:16', '1:5', '3:6', '4:1', '2:27', '13:4', '3:5', '33:27', '2:14', '2:15', '1:37:48', '7:35', '30', '30', '31', '47', '2:26', '1:48', '14:41', '3:7', '2:49

In [ ]:
#more shitfixing
duration=[]
duration_sec=[]
for d in duration1:
  t=d.split(":")
  if len(t)==2:
    minute=t[0]
    second=t[1]
    hour=00
  else:
    hour=t[0]
    hr=t[0]
    minute=t[1]
    second=t[2]
  if int(second)<10:
    second="0"+second
  if int(minute)<10:
    minute="0"+minute
  duration.append(str(hour)+":"+minute+":"+second)
  print(minute)
  duration_sec.append((int(second)+int(minute)*60+int(hour)*3600))

In [ ]:
duration_sec

In [ ]:
df["duration"]=duration
df["duration_sec"]=duration_sec
df.sort_values(by="duration_sec")

In [36]:
import numpy as np
df_duration=df[df["duration_sec"]<600]
bins = np.linspace(df_duration.duration_sec.min(), df_duration.duration_sec.max(), 30)


In [39]:
df["views"]=df["views"].astype(int)
groups = df_duration.groupby(np.digitize(df_duration.duration_sec, bins))

In [48]:
optimal_duration=groups.mean()

In [50]:
optimal_duration.to_csv("optimal_duration.csv")

### Days of the week Videos do best

In [56]:
df["dayoftheweek"]=df["date"].dt.day_name()

In [59]:
optimal_day=df.groupby(by="dayoftheweek").mean()

In [61]:
optimal_day.to_csv("optimal_day.csv")

In [105]:
optimal_day

,views,duration_sec
dayoftheweek,,
Friday,1.225501e+06,334.187500
Monday,1.014532e+06,289.130682
Saturday,7.338042e+05,388.891892
Sunday,1.008322e+06,269.576271
Thursday,5.863760e+05,274.414530
Tuesday,1.219512e+06,190.917949
Wednesday,3.270312e+06,222.824074


### Best Time to Post

In [65]:
df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.hour

In [86]:
bins = np.linspace(df.time.min(), df.time.max(), 24)
groups = df.groupby(np.digitize(df.time, bins))

In [88]:
optimal_time=groups.mean()
optimal_time.to_csv("optimal_time.csv")

### Analysis on Video Tags
Top 100 Tags

In [106]:
df.explode("tags").groupby("tags").mean().sort_values("views",ascending=False)

,views,time,duration_sec
tags,,,
Drums,1.065618e+08,8.000000,147.800000
Shows for kids,9.205445e+07,8.285714,147.428571
Jungle,7.731177e+07,8.000000,151.000000
Mighty Little Bheem,7.324198e+07,9.222222,157.111111
Kids,6.780417e+07,6.888889,149.000000
...,...,...,...
Millie Brown,6.110000e+02,19.000000,521.000000
Ross Duffer,6.110000e+02,19.000000,521.000000
Charlie Heaton,6.110000e+02,19.000000,521.000000


In [104]:
top100tags.to_csv("top100tags.csv")